# Predicting Major Power Outages

**Jiawei Yang, Zhixing Jiang**

## Summary of Findings


### Introduction
TODO

### Baseline Model
TODO

### Final Model
TODO

### Fairness Analysis
TODO

### Introduction
Based on the dataset of major power outages happened in the United State during recent years, our ultimate goal in thsi project is to predict the durtaion of the power outage given various characteristics that pertain to directly to the specific instance of power outage and indirectly to the area where the outage takes place.

We chose the response variable to be `outage.duration`: a continous varaible with in the unit of minutes. We chose it since it accurately quantify the length of duration of outages and is given in the data set (compared to other possible option such as duration recorded in days, which means a drop in granularity of our model)

Since we attempt to predict a continous outcome, we use regression-based algorithm to build the model: we end up using decision tree regression after comparing it to KNN regression and linear regression. Therefore, we use two main metrics to evaluate the performance of our model: Mean Squared Error and R-squared.

### Baseline Model
In the prediction process, we assume that the model is used to predict the duration of a power outage right after the happening of that power outage. Therefore the following features from the original data set could not be used as input variables, since they are by natural unobserved at the point of outage happenings:
1. The end time of outage: this directly tells you the outage durtaion.
2. The number of customer affacted: we assume this statistic is collected/estiamted at least some time after one outage happening.

We include the following features in the model:
1. State: the state in which the outage took place, originally recorded as categorical strings with 50+ levels. (One-hot encoded in the model)
2. Cause: the description of cause of outage, originally recorded as categorical stirngs with 20+ levels. (One-hot encoded in the model) One may argue the cause or reason of the outage is not revealed immediately after the outage happening. (Since it may take time for the company to investigate) We are aware of such potential draw back, but decided to include this feature in the model assuming the company would know right away why an outage happens.
3. com price: contionus
... (补充)

We use train-test split and cross validation to pick the model that maximize both in-sample and out-of-sample performance: specifically, we pick the best hyperparameter, max depth, of the prediction tree based on R-sqaured.

### Final Model
On top of the features present in the baseline model, we include the following three features in the final model, hoping to improve performance. The additional features are results of feature engineering based on both information recorded in the model and additional data we collected (during project 3). Feature engineering is done by `ColumnTransformer` classes we define.

We believe that extreme weather could hinder the process of restoring a power outage and therefore influence the duration of power outage. We further specify 'extreme weather' to be hot summer and cold winder by encoding the power outages with the following two features.

1. `hot_summer`: a binary encoding, `True` if the power outage took place both:
<br>
$\;\;\;\;\;\;$   in one of the months: June, July or August
<br>
$\;\;\;\;\;\;$   in a state where the average summer temperature is above xx degree celsius.
2. `cold_winter`: a  binary encoding, `True` if the power outage took place both:
<br>
$\;\;\;\;\;\;$   in one of the months: December, January or February
<br>
$\;\;\;\;\;\;$   in a state where the average winter temperature is below 0 degree celsius.

During our exploratory 

## Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Load and Cleaning

In [413]:
#Convert all column names to lower case
outage_org = pd.read_excel('outage.csv')
outage_org.columns = [t.lower() for t in outage_org.columns]
outage_org

,obs,year,month,u.s._state,postal.code,nerc.region,climate.region,anomaly.level,climate.category,outage.start.date,...,poppct_urban,poppct_uc,popden_urban,popden_uc,popden_rural,areapct_urban,areapct_uc,pct_land,pct_water_tot,pct_water_inland
0,1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
1,2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
2,3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
3,4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
4,5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1531,1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256
1532,1533,2009,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,2009-08-29,...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256


In [6]:
def remove_na(df):
    return df.loc[~df.isna().any(axis = 1).values]

In [399]:
#Cleaning the dataframe: Extract only three columns: u.s._state, cause.category, and outage.duration
#                        Remove all nans rows from the dataframe
#                        Reset the index
selected_cols = ['u.s._state', 'cause.category', 'outage.duration']
sub = outage[selected_cols]
sub = remove_na(sub)
new_index = np.arange(sub.shape[0])
sub['new_index'] = new_index
sub = sub.set_index('new_index')
sub

,u.s._state,cause.category,outage.duration
new_index,,,
0,Minnesota,severe weather,3060.0
1,Minnesota,intentional attack,1.0
2,Minnesota,severe weather,3000.0
3,Minnesota,severe weather,2550.0
4,Minnesota,severe weather,1740.0
...,...,...,...
1471,Idaho,intentional attack,0.0
1472,Idaho,system operability disruption,220.0
1473,North Dakota,public appeal,720.0


### Baseline Model

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier

In [401]:
#Build the model by transforming u.s._state and cause.category columns using one hot encoder from sklearn
#Fit the data using DecisionTreeRegressor with max_depth = 5 and find the R-square of the prediction
prepro = ColumnTransformer(
    transformers = [('ohe', OneHotEncoder(handle_unknown='ignore'), ['u.s._state', 'cause.category'])],
    remainder = 'passthrough'
)
ppl_base = Pipeline(
    steps = [
        ('prepro', prepro),
        ('dtree', DecisionTreeRegressor(max_depth = 5))
    ]
)
ppl_base.fit(sub.drop(columns = ['outage.duration']), sub['outage.duration'])
ppl_base.score(sub.drop(columns = ['outage.duration']), sub['outage.duration'])

0.30606349925606213

In [441]:
#Dictionary of hyper parameter used to find the best fit
hypers = {
    'dtree__max_depth':[2,3,4,5,6,7,8,9,10,11,12,13,14,15]
}

In [448]:
#Apply GridSearchCV on the baseline model to find the best-fit hyperparameter
X_train, X_test, y_train, y_test = train_test_split(
    sub.drop(columns = ['outage.duration']),
    sub['outage.duration'],)
searcher = GridSearchCV(ppl_base, hypers, cv = 5)
searcher.fit(X_train, y_train)
searcher.best_params_, searcher.score(X_train, y_train), searcher.score(X_test, y_test)

({'dtree__max_depth': 2}, 0.19253851967476987, 0.18618847547276773)

### Final Model

In [373]:
#Define a QuantileBinarizerByGroup() class that would transform the duration time that exceeds 0.90 quantile by state
#to True and under 0.90 quantile by state to False

from sklearn.base import BaseEstimator, TransformerMixin

class QuantileBinarizerByGroup(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X might not be a pandas DataFrame (e.g. a np.array)
        df = pd.DataFrame(X)
        self.grps_ = df.groupby('u.s._state')['outage.duration'].quantile(0.90).to_dict()
        return self
    def transform(self, X, y=None):

        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        

        df = pd.DataFrame(columns = ['outage.duration'])
        for state in self.grps_.keys():
            def binarizer(duration):
                if duration > grps[state]:
                    return True
                else:
                    return False
            ser = (X[X['u.s._state'] == state]['outage.duration'].apply(binarizer))
            temp_df = pd.DataFrame(ser)
            df = pd.concat([df, temp_df])
        return df.sort_index()

In [379]:
#Transform the outage_duration columns and store it to a col
prepro_pre = ColumnTransformer(
    transformers = [('QuantileBinarizerByGroup', QuantileBinarizerByGroup(),['u.s._state','outage.duration'])],
    remainder = 'passthrough'
)
ppl_pre = Pipeline(
    [('prepre', prepro_pre)]
)
col = pd.DataFrame(ppl_pre.fit_transform(sub))[0]
col

0       False
1       False
2       False
3       False
4       False
        ...  
1471    False
1472    False
1473    False
1474    False
1475     True
Name: 0, Length: 1476, dtype: object

In [449]:
#Import temp_state.csv which contains the state name, the average temperature in winter by state, 
#and the average temperature in summer by state
temp_df = pd.read_csv('temp_state.csv')
temp_df

,u.s._state,winter_avg_temp,summer_avg_temp
0,Alabama,8.10,25.90000
1,Alaska,-16.30,11.30000
2,Arizona,6.40,25.60000
3,Arkansas,5.30,26.00000
4,California,7.90,23.00000
5,Colorado,-3.40,18.40000
6,Connecticut,-1.90,20.70000
7,Delaware,2.30,23.40000
8,Florida,15.20,27.20000
9,Georgia,8.80,25.90000


In [391]:
#Store col to columns long_by_state in final_sub and merge temp_df to the final_sub 
final_sub = sub.copy()
final_sub['long_by_state'] = col
final_sub = final_sub.merge(temp_df, left_on = 'u.s._state', right_on = 'u.s._state')
final_sub

,u.s._state,cause.category,outage.duration,long_by_state,winter_avg_temp,summer_avg_temp
0,Minnesota,severe weather,3060.0,False,-10.9,19.3
1,Minnesota,intentional attack,1.0,False,-10.9,19.3
2,Minnesota,severe weather,3000.0,False,-10.9,19.3
3,Minnesota,severe weather,2550.0,False,-10.9,19.3
4,Minnesota,severe weather,1740.0,False,-10.9,19.3
...,...,...,...,...,...,...
1471,Idaho,intentional attack,0.0,False,-3.7,17.6
1472,Idaho,system operability disruption,220.0,False,-3.7,17.6
1473,North Dakota,public appeal,720.0,False,-11.0,19.2
1474,South Dakota,islanding,59.0,False,-6.9,21.1


In [392]:
#Function that converts summer temperature into boolean to define whether the summer is hot
def transform_hot(temp):
    df = pd.DataFrame(temp)
    def check_warm(x):
        if x >= 25:
            return True
        else:
            return False
    return pd.DataFrame(df['summer_avg_temp'].apply(check_warm))

In [393]:
#Function that converts winter temperature into boolean to define whether the winter is cold
def transform_cold(temp):
    df = pd.DataFrame(temp)
    def check_cold(x):
        if x <= 0 :
            return True
        else:
            return False
    return pd.DataFrame(df['winter_avg_temp'].apply(check_cold))

In [386]:
#Build the model by transforming u.s._state, cause.category, 
#and long_by_state columns using one hot encoder from sklearn
#Fit the data using DecisionTreeRegressor with max_depth = 5 and find the R-square of the prediction
prepro = ColumnTransformer(
    transformers = [('ohe', OneHotEncoder(handle_unknown='ignore'), ['u.s._state', 'cause.category','long_by_state']),
                   ('cold_changed_prepro', cold_changed, ['winter_avg_temp']),
                   ('hot_changed_prepro', hot_changed, ['summer_avg_temp'])],
    remainder = 'passthrough'
)
ppl_improve = Pipeline(
    steps = [
        ('prepro', prepro),
        ('dtree2', DecisionTreeRegressor(max_depth = 5))
        
    ]
)

ppl_improve.fit(final_sub.drop(columns = ['outage.duration']), final_sub['outage.duration'])
ppl_improve.score(final_sub.drop(columns = ['outage.duration']), final_sub['outage.duration'])

0.6324564463906786

In [437]:
#Dictionary of hyper parameter used to find the best fit
hypers2 = {
    'dtree2__max_depth':[2,3,4,5,6,7,8,9,10,11,12,13,14,15],
    'dtree2__splitter': ['best', 'random']
}

In [440]:
#Apply GridSearchCV on the baseline model to find the best-fit hyperparameter
X2_train, X2_test, y2_train, y2_test = train_test_split(final_sub.drop(columns = ['outage.duration']),
                                                    final_sub['outage.duration'])
searcher = GridSearchCV(ppl_improve, hypers2, cv = 5)
searcher.fit(X2_train, y2_train)
searcher.best_params_, searcher.score(X2_train, y2_train), searcher.score(X2_test, y2_test)

({'dtree2__max_depth': 14, 'dtree2__splitter': 'best'},
 0.7448335677976139,
 0.5201515729578026)

### Fairness Analysis

Null Hypothesis: Our model is fair. The R-Squared for the lower 90 percentile is roughly the same as the R-Squared of the top 10 percentile in outage duration by state. <br>
Alternative Hypothesis: Our model is unfair. The R-Squared for the lower 90 percentile is higher than the R-Squared of the top 10 percentile in outage duration by state.

In [453]:
final_sub

,u.s._state,cause.category,outage.duration,long_by_state,winter_avg_temp,summer_avg_temp
0,Minnesota,severe weather,3060.0,False,-10.9,19.3
1,Minnesota,intentional attack,1.0,False,-10.9,19.3
2,Minnesota,severe weather,3000.0,False,-10.9,19.3
3,Minnesota,severe weather,2550.0,False,-10.9,19.3
4,Minnesota,severe weather,1740.0,False,-10.9,19.3
...,...,...,...,...,...,...
1471,Idaho,intentional attack,0.0,False,-3.7,17.6
1472,Idaho,system operability disruption,220.0,False,-3.7,17.6
1473,North Dakota,public appeal,720.0,False,-11.0,19.2
1474,South Dakota,islanding,59.0,False,-6.9,21.1


In [539]:
state = final_sub['u.s._state'].unique()
state

array(['Minnesota', 'Tennessee', 'Wisconsin', 'West Virginia', 'Michigan',
       'Texas', 'Indiana', 'Alabama', 'Mississippi', 'Illinois',
       'Washington', 'Arizona', 'Maryland', 'Pennsylvania', 'Kentucky',
       'Utah', 'Ohio', 'North Carolina', 'New Jersey', 'South Carolina',
       'Oregon', 'District of Columbia', 'Missouri', 'Delaware', 'Iowa',
       'Montana', 'New York', 'Louisiana', 'Florida', 'California',
       'Arkansas', 'Virginia', 'Nebraska', 'Wyoming', 'New Mexico',
       'Vermont', 'Georgia', 'Connecticut', 'Oklahoma', 'Massachusetts',
       'Maine', 'New Hampshire', 'Nevada', 'Colorado', 'Kansas', 'Hawaii',
       'Idaho', 'North Dakota', 'South Dakota'], dtype=object)

In [556]:
[2,3]+list((np.array([211])))

[2, 3, 211]

In [607]:
shuffled_lst = pd.Series([],dtype=object)
for x in state:
    shuffled_lst = shuffled_lst.append(pd.Series(np.random.permutation(final_sub[final_sub['u.s._state'] == x]['long_by_state'])), ignore_index = True)

In [608]:
shuffled_lst

0       False
1       False
2       False
3       False
4       False
        ...  
1471     True
1472    False
1473    False
1474    False
1475     True
Length: 1476, dtype: object

In [535]:
final_sub_copy = final_sub.copy()
diff_lst = []
for _ in range(100):
    shuffled_lst = pd.Series([],dtype=object)
    for x in state:
        #Shuffle the long_by_state columns within ea
        shuffled_lst = shuffled_lst.append(pd.Series(np.random.permutation(final_sub
                                            [final_sub['u.s._state'] == x]['long_by_state'])), ignore_index = True)
    copy_false = final_sub_copy[final_sub_copy['long_by_state'] == False]
    ppl_improve.fit(copy_false.drop(columns = ['outage.duration']), copy_false['outage.duration'])
    false_score = ppl_improve.score(copy_false.drop(columns = ['outage.duration']), copy_false['outage.duration'])

    copy_true = final_sub_copy[final_sub_copy['long_by_state'] == True]
    ppl_improve.fit(copy_true.drop(columns = ['outage.duration']), copy_true['outage.duration'])
    true_score = ppl_improve.score(copy_true.drop(columns = ['outage.duration']), copy_true['outage.duration'])
    diff_lst.append(false_score - true_score)

In [536]:
p_val = np.mean([np.array(diff_lst) > 0])
p_val

0.043